In [1]:
import pygame
import sys
import random
import time
import os

# Khởi tạo pygame
pygame.init()
pygame.mixer.init()

# Màu sắc
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
BLUE = (65, 105, 225)  # Royal Blue
RED = (220, 20, 60)    # Crimson
GREEN = (34, 139, 34)  # Forest Green
GRAY = (200, 200, 200)
GOLD = (255, 215, 0)   # Gold cho hiệu ứng
PURPLE = (128, 0, 128) # Purple cho nền

# Thiết lập màn hình
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Memory Game")

# Font chữ
try:
    font = pygame.font.Font(None, 36)
    small_font = pygame.font.Font(None, 24)
    title_font = pygame.font.Font(None, 60)
except:
    font = pygame.font.SysFont(None, 36)
    small_font = pygame.font.SysFont(None, 24)
    title_font = pygame.font.SysFont(None, 60)

# Âm thanh
try:
    correct_sound = pygame.mixer.Sound("sounds\\correct-156911.mp3")
    wrong_sound = pygame.mixer.Sound("sounds\\buzzer-or-wrong-answer-20582.mp3")
    level_up_sound = pygame.mixer.Sound("sounds\\scale-e6-14577.mp3")
except:
    # Nếu không tìm thấy file âm thanh, tạo âm thanh đơn giản
    correct_sound = pygame.mixer.Sound(pygame.sndarray.make_sound(pygame.Surface((1, 1))))
    wrong_sound = pygame.mixer.Sound(pygame.sndarray.make_sound(pygame.Surface((1, 1))))
    level_up_sound = pygame.mixer.Sound(pygame.sndarray.make_sound(pygame.Surface((1, 1))))

# Tạo gradient background
def create_gradient_background(top_color, bottom_color):
    background = pygame.Surface((WIDTH, HEIGHT))
    for y in range(HEIGHT):
        # Tính toán màu cho từng dòng pixel
        r = top_color[0] + (bottom_color[0] - top_color[0]) * y / HEIGHT
        g = top_color[1] + (bottom_color[1] - top_color[1]) * y / HEIGHT
        b = top_color[2] + (bottom_color[2] - top_color[2]) * y / HEIGHT
        pygame.draw.line(background, (r, g, b), (0, y), (WIDTH, y))
    return background

background = create_gradient_background((70, 130, 180), (25, 25, 112))  # SteelBlue to MidnightBlue

class Circle:
    def __init__(self, x, y, radius, number):
        self.x = x
        self.y = y
        self.radius = radius
        self.number = number
        self.show_number = True
        self.clicked = False
        self.color = BLUE
        self.shine = 0  # Hiệu ứng lấp lánh
    
    def draw(self):
        # Vẽ viền ngoài
        if self.shine > 0:
            pygame.draw.circle(screen, GOLD, (self.x, self.y), self.radius + 5)
            self.shine -= 1
        
        # Vẽ ô tròn với hiệu ứng 3D
        pygame.draw.circle(screen, BLACK, (self.x + 3, self.y + 3), self.radius)  # Bóng đổ
        pygame.draw.circle(screen, self.color, (self.x, self.y), self.radius)
        
        # Vẽ viền trong
        pygame.draw.circle(screen, WHITE, (self.x, self.y), self.radius - 5, 2)
        
        if self.show_number:
            text = font.render(str(self.number), True, WHITE)
            text_rect = text.get_rect(center=(self.x, self.y))
            screen.blit(text, text_rect)
    
    def is_clicked(self, pos):
        distance = ((pos[0] - self.x) ** 2 + (pos[1] - self.y) ** 2) ** 0.5
        return distance <= self.radius

class Button:
    def __init__(self, x, y, width, height, text, color):
        self.rect = pygame.Rect(x, y, width, height)
        self.text = text
        self.color = color
        self.hover_color = (min(color[0] + 30, 255), min(color[1] + 30, 255), min(color[2] + 30, 255))
        self.active_color = color
    
    def draw(self):
        mouse_pos = pygame.mouse.get_pos()
        if self.rect.collidepoint(mouse_pos):
            pygame.draw.rect(screen, self.hover_color, self.rect, border_radius=10)
            # Vẽ hiệu ứng sáng
            pygame.draw.rect(screen, WHITE, self.rect, width=2, border_radius=10)
        else:
            pygame.draw.rect(screen, self.active_color, self.rect, border_radius=10)
        
        text_surface = font.render(self.text, True, WHITE)
        text_rect = text_surface.get_rect(center=self.rect.center)
        screen.blit(text_surface, text_rect)
    
    def is_clicked(self, pos):
        return self.rect.collidepoint(pos)

class Game:
    def __init__(self):
        self.level = 1
        self.score = 0
        self.high_score = self.load_high_score()
        self.circles = []
        self.show_numbers = True
        self.current_number = 1
        self.game_state = "menu"  # menu, ready, memorize, playing, won, lost
        self.message = "Start new game!"
        self.start_time = 0
        self.level_complete_time = 0  # Thời điểm hoàn thành level
        
        # Tạo các nút
        self.start_button = Button(WIDTH // 2 - 100, HEIGHT // 2, 200, 60, "Start", BLUE)
        self.menu_button = Button(WIDTH // 2 - 100, HEIGHT // 2 + 100, 200, 60, "Menu", PURPLE)
        self.particles = []  # Hệ thống particle
    
    def load_high_score(self):
        try:
            with open("high_score.txt", "r") as file:
                return int(file.read())
        except:
            return 0
    
    def save_high_score(self):
        try:
            with open("high_score.txt", "w") as file:
                file.write(str(self.high_score))
        except:
            pass
    
    def start_game(self):
        self.circles = []
        self.current_number = 1
        self.game_state = "memorize"
        self.message = "Remember the positions!"
        self.particles = []
        
        # Tạo n ô tròn với n = level + 2
        n = self.level + 2
        positions = []
        
        # Đảm bảo các ô tròn không chồng lên nhau
        radius = 40
        margin = 20
        max_attempts = 100
        
        for i in range(1, n + 1):
            attempts = 0
            while attempts < max_attempts:
                x = random.randint(radius + margin, WIDTH - radius - margin)
                y = random.randint(radius + margin, HEIGHT - 100 - radius - margin)
                
                # Kiểm tra xem vị trí có chồng lên vị trí đã có không
                valid_position = True
                for pos in positions:
                    distance = ((pos[0] - x) ** 2 + (pos[1] - y) ** 2) ** 0.5
                    if distance < 2 * radius + margin:
                        valid_position = False
                        break
                
                if valid_position:
                    positions.append((x, y))
                    self.circles.append(Circle(x, y, radius, i))
                    break
                
                attempts += 1
            
            # Nếu không tìm được vị trí hợp lệ sau nhiều lần thử, đặt vị trí mặc định
            if attempts >= max_attempts:
                x = random.randint(radius + margin, WIDTH - radius - margin)
                y = random.randint(radius + margin, HEIGHT - 100 - radius - margin)
                positions.append((x, y))
                self.circles.append(Circle(x, y, radius, i))
        
        self.show_numbers = True
        self.start_time = time.time()
    
    def hide_numbers(self):
        self.show_numbers = False
        for circle in self.circles:
            circle.show_number = False
        self.game_state = "playing"
        self.message = "Click the buttun in ascending order!"
    
    def add_particles(self, x, y, color):
        # Thêm hiệu ứng particle
        for _ in range(20):
            particle = {
                'x': x,
                'y': y,
                'dx': random.uniform(-3, 3),
                'dy': random.uniform(-3, 3),
                'radius': random.uniform(2, 5),
                'color': color,
                'life': 30
            }
            self.particles.append(particle)
    
    def update_particles(self):
        # Cập nhật và vẽ các particle
        for particle in self.particles[:]:
            particle['x'] += particle['dx']
            particle['y'] += particle['dy']
            particle['life'] -= 1
            
            if particle['life'] <= 0:
                self.particles.remove(particle)
                continue
                
            alpha = min(255, int(255 * (particle['life'] / 30)))
            color = particle['color'] + (alpha,)
            pygame.draw.circle(screen, color, (int(particle['x']), int(particle['y'])), int(particle['radius']))
    
    def handle_click(self, pos):
        if self.game_state == "menu":
            if self.start_button.is_clicked(pos):
                self.level = 1
                self.score = 0
                self.game_state = "ready"
                self.start_game()
        elif self.game_state == "lost":
            if self.menu_button.is_clicked(pos):
                self.game_state = "menu"
        elif self.game_state == "playing":
            for circle in self.circles:
                if circle.is_clicked(pos) and not circle.clicked:
                    if circle.number == self.current_number:
                        # Đúng
                        circle.clicked = True
                        circle.color = GREEN
                        circle.show_number = True
                        circle.shine = 15  # Bắt đầu hiệu ứng lấp lánh
                        pygame.mixer.Sound.play(correct_sound)
                        self.add_particles(circle.x, circle.y, GREEN)
                        self.current_number += 1
                        self.score += 10 * self.level
                        
                        # Kiểm tra xem đã hoàn thành level chưa
                        if self.current_number > len(self.circles):
                            self.game_state = "won"
                            self.message = f"Well done! You finished level: {self.level}!"
                            self.level_complete_time = time.time()
                            pygame.mixer.Sound.play(level_up_sound)
                            # Kiểm tra high score
                            if self.score > self.high_score:
                                self.high_score = self.score
                                self.save_high_score()
                    else:
                        # Sai
                        circle.color = RED
                        circle.show_number = True
                        pygame.mixer.Sound.play(wrong_sound)
                        self.add_particles(circle.x, circle.y, RED)
                        self.game_state = "lost"
                        self.message = f"Game over! Your score: {self.score}"
                        # Kiểm tra high score
                        if self.score > self.high_score:
                            self.high_score = self.score
                            self.save_high_score()
                    break
    
    def check_next_level(self):
        # Kiểm tra xem đã đến lúc chuyển level chưa
        if self.game_state == "won" and time.time() - self.level_complete_time >= 0.5:
            self.level += 1
            self.start_game()

def main():
    clock = pygame.time.Clock()
    game = Game()
    
    running = True
    while running:
        screen.blit(background, (0, 0))
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.MOUSEBUTTONDOWN:
                game.handle_click(event.pos)
        
        # Ẩn số sau 1 giây (tăng lên từ 1 giây)
        if game.game_state == "memorize" and time.time() - game.start_time > 1:
            game.hide_numbers()
        
        # Kiểm tra chuyển level sau 1.5 giây khi hoàn thành level
        if game.game_state == "won":
            game.check_next_level()
        
        # Vẽ các ô tròn
        for circle in game.circles:
            circle.draw()
        
        # Cập nhật và vẽ particles
        game.update_particles()
        
        # Hiển thị thông tin
        if game.game_state != "menu":
            # Vẽ hộp thông tin
            info_box = pygame.Rect(10, HEIGHT - 90, 200, 80)
            pygame.draw.rect(screen, (0, 0, 0, 128), info_box, border_radius=5)
            pygame.draw.rect(screen, WHITE, info_box, width=2, border_radius=5)
            
            level_text = small_font.render(f"Level: {game.level}", True, WHITE)
            screen.blit(level_text, (20, HEIGHT - 80))
            
            score_text = small_font.render(f"Sroce: {game.score}", True, WHITE)
            screen.blit(score_text, (20, HEIGHT - 60))
            
            high_score_text = small_font.render(f"Record: {game.high_score}", True, WHITE)
            screen.blit(high_score_text, (20, HEIGHT - 40))
        
        # Hiển thị thông báo
        if game.message:
            message_bg = pygame.Rect(WIDTH // 2 - 300, HEIGHT - 60, 600, 50)
            pygame.draw.rect(screen, (0, 0, 0, 180), message_bg, border_radius=10)
            pygame.draw.rect(screen, WHITE, message_bg, width=2, border_radius=10)
            
            message_text = font.render(game.message, True, WHITE)
            message_rect = message_text.get_rect(center=(WIDTH // 2, HEIGHT - 35))
            screen.blit(message_text, message_rect)
        
        # Hiển thị các nút và giao diện khác
        if game.game_state == "menu":
            # Vẽ tiêu đề
            title_shadow = title_font.render("Memory Game", True, BLACK)
            title_text = title_font.render("Memory Game", True, GOLD)
            
            screen.blit(title_shadow, (WIDTH // 2 - title_shadow.get_width() // 2 + 3, 
                                    HEIGHT // 4 - title_shadow.get_height() // 2 + 3))
            screen.blit(title_text, (WIDTH // 2 - title_text.get_width() // 2, 
                                  HEIGHT // 4 - title_text.get_height() // 2))
            
            # Vẽ nút bắt đầu
            game.start_button.draw()
            
            # Hiển thị hướng dẫn
            instruction = small_font.render("Remember", True, WHITE)
            instruction_rect = instruction.get_rect(center=(WIDTH // 2, HEIGHT // 2 + 150))
            screen.blit(instruction, instruction_rect)
        
        elif game.game_state == "lost":
            game.menu_button.draw()
        
        pygame.display.flip()
        clock.tick(60)
    
    pygame.quit()
    sys.exit()

if __name__ == "__main__":
    main()

pygame 2.6.1 (SDL 2.28.4, Python 3.11.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

d:\Anaconda3\envs\Linhtinh\Lib\site-packages\IPython\core\interactiveshell.py:3675: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
